In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=35ab5e8b283efc733197f856d6ddc280e327728defa40aed1e9433465c30ac3f
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [ ]:

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Softmax
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# GPU 사용
device = torch.device("cuda:0")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [ ]:
tokenizer_e = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer_d = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer_e.vocab_file, padding_token='[PAD]')

## 사용자 정의 토큰 추가
new_tokens = ['우울', '슬픔', '외로움', '분노', '무기력', '의욕상실', '식욕저하', '식욕증가', '감정조절이상',
              '불면', '초조', '피로', '죄책감', '집중력저하', '자신감 저하', '자존감저하', '절망', '자살', '불안']
tokenizer_d.add_tokens(new_tokens)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


19

In [ ]:
# 필요한 클래스

class BERTSentenceTransform:

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab ##추가

    def __call__(self, line):


        # 유니코드로 변환
        text_a = line[0]
        if self._pair:
            assert len(line) == 2  # 2개로 이뤄진문장 있을 수 있음3
            text_b = line[1]

        #tokens_a = self._tokenizer(text_a)
        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # `tokens_a` 와 `tokens_b` max_seq_length-3을 넘지 않도록 수정
            # [CLS], [SEP], [SEP]를 포함하므로 "- 3" 들어가야함
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # 문장 한 개일 경우 [CLS], [SEP] 두 개이므로 "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]


        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        #input_ids = tokens_a['input_ids']

        # 문장의 유효길이 valid_length는 input_ids의 길이. 그 이외는 0 패딩
        valid_length = len(input_ids)

        if self._pad:
            # 시퀀스 길이만큼 0 패딩
            padding_length = self._max_seq_length - valid_length
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


#데이터셋 input_ids, valid_length, segment_ids 로
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 num_classes,
                 hidden_size = 768,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 파라미터
max_len = 64
batch_size = 16
warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
depress_model = torch.load('/content/drive/MyDrive/KOBERT/model/kobert_text_depress_model_batch16_LR5e-05.pt', map_location=torch.device('cpu'))
depress_model.eval()
print(depress_model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8021, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [ ]:
emotion_model = torch.load('/content/drive/MyDrive/KOBERT/model/kobert_emotion_model_LR5e-05_batch16.pt', map_location=torch.device('cpu'))
emotion_model.eval()
print(emotion_model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

# 사용자 입력 전처리

In [ ]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.3-py3-none-any.whl size=54258 sha256=5d024e8e645e9f998b649fd7ecb22e1bc5c6e5333c590d35a338fd7fa1948dc2
  Stored in directory: /root/.cache/pip/wheels/d8/9e/a3/5b09e3f14722fa0d77f47fe840668d426760023bdd11b0fbd9
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646666 sha256=dbcbb54c020e88e2583a4a5993033eb1841f14272c0d908b9dd7aa2d19e92c1d
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d
Successfully built kss pecab


# 사용자 입력을 분석하여 감정 상태를 추론하고, 감정에 대한 점수를 반환하는 함수.


In [ ]:
"""
사용자 입력을 분석하여 감정 상태를 추론하고, 감정에 대한 점수를 반환하는 함수.
"""
from torch.nn import Softmax
from kss import split_sentences

def model_output(model, user_input, tokenizer):

    predict_sentences = split_sentences(user_input)

    # 사용자 데이터셋 생성
    user_dataset = [[sentence, '0']  for sentence in predict_sentences]
    print(user_dataset)

    # logits 리스트 초기화
    logits_list = []

    # BERT 데이터셋 준비
    user_dataset_ids = BERTDataset(user_dataset, 0, 1, tokenizer, vocab, max_len, True, False)
    user_dataset_dataloader = torch.utils.data.DataLoader(user_dataset_ids, batch_size=batch_size, num_workers=2)

    # 모델을 사용하여 예측값 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(user_dataset_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length.to(device)

        output = model(token_ids, valid_length, segment_ids)

        # 출력값 저장
        for logits in output:
            logits = logits.detach().cpu().numpy()
            logits_list.append(logits)

    return logits_list


## 우울 모델 함수
def Analyze_fn_depress(logits_list):

    depress_list = []

    # 감정 사전
    depress_dict = {0: '감정조절이상', 1: '불면', 2: '분노', 3: '불안', 4: '초조', 5: '슬픔', 6: '외로움', 7: '우울', 8: '의욕상실', 9: '무기력',
                10: '자살', 11: '자존감저하', 12: '절망', 13: '죄책감', 14: '집중력저하', 15: '피로', 16: '식욕저하', 17: '식욕증가', 18: '일상'}

    # softmax 점수 계산
    for logits in logits_list:
        softmax = Softmax(dim=0)
        score = softmax(torch.tensor(logits))  # 음성과 함께 사용할 예정
        max_value, max_index = torch.max(score, dim=0)

        emotion = depress_dict[int(max_index)]

        # 50% 이하인 경우 '일상'으로 지정 - 애매한 문장은 중립
        if max_value.item() <= 0.6:
            print("emotion :", emotion, "일 확률", round(100 * (max_value.item()), 2), "%")
            emotion = '일상'

        else :
            print("emotion :", emotion, "일 확률", round(100 * (max_value.item()), 2), "%")

        depress_list.append(emotion)


    if (depress_list.count('일상')) != len(depress_list):
        depress_value = round((len(depress_list) - depress_list.count('일상')) / len(depress_list), 2)
    else :
        depress_value = 0.0

    return [depress_list, depress_value]



## 감정 모델 함수
def Analyze_fn_emotion(logits_list):

    emotion_list = []
    emotion_dict = {0: '두려움', 1: '놀람', 2: '화남', 3: '슬픔', 4: '중립', 5: '행복함', 6: '모멸감'}

    for logits in logits_list:
        emotion = emotion_dict[np.argmax(logits)]
        emotion_list.append(emotion)
    print(emotion_list)
    return emotion_list


In [ ]:
## test1. 우울 문장 있을 경우
user_input = "너무 우울하다 눈물이 자꾸 난다 정말 죽고싶다는 생각이 자꾸 든다 요즘 나의 인간관계가 복잡하다 친한 친구와의 관계가 어색해지고 가족들과의 대화도 어색해진다 내가 무슨 말을 해도 상대방이 이해하지 못하는 것 같아서 우울하다 나는 왜 이렇게 다른 사람과의 대화가 어려운 걸까 내가 잘못한 걸까요즘 스트레스가 많이 쌓여서 힘들다 일과 공부 때문에 시간이 부족하고 여러 가지 일들이 겹쳐져서 정말 지친다 어쩌면 좋을까 휴식을 취하면서 스트레스를 풀 수 있는 방법을 찾아봐야겠다 혹시 좋은 조언이 있을까"
depress = Analyze_fn_depress(model_output(depress_model ,user_input, tokenizer_d))
emotion = Analyze_fn_emotion(model_output(emotion_model ,user_input, tokenizer_e))
print(depress)
print("\n\n\n")
print(emotion)

For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



[['너무 우울하다 눈물이 자꾸 난다', '0'], ['정말 죽고싶다는 생각이 자꾸 든다 요즘 나의 인간관계가 복잡하다 친한 친구와의 관계가 어색해지고 가족들과의 대화도 어색해진다', '0'], ['내가 무슨 말을 해도 상대방이 이해하지 못하는 것 같아서 우울하다', '0'], ['나는 왜 이렇게 다른 사람과의 대화가 어려운 걸까 내가 잘못한 걸까요즘 스트레스가 많이 쌓여서 힘들다 일과 공부 때문에 시간이 부족하고 여러 가지 일들이 겹쳐져서 정말 지친다 어쩌면 좋을까 휴식을 취하면서 스트레스를 풀 수 있는 방법을 찾아봐야겠다', '0'], ['혹시 좋은 조언이 있을까', '0']]
emotion : 우울 일 확률 97.77 %
emotion : 자살 일 확률 93.94 %
emotion : 우울 일 확률 95.26 %
emotion : 피로 일 확률 92.05 %
emotion : 분노 일 확률 69.02 %
[['너무 우울하다 눈물이 자꾸 난다', '0'], ['정말 죽고싶다는 생각이 자꾸 든다 요즘 나의 인간관계가 복잡하다 친한 친구와의 관계가 어색해지고 가족들과의 대화도 어색해진다', '0'], ['내가 무슨 말을 해도 상대방이 이해하지 못하는 것 같아서 우울하다', '0'], ['나는 왜 이렇게 다른 사람과의 대화가 어려운 걸까 내가 잘못한 걸까요즘 스트레스가 많이 쌓여서 힘들다 일과 공부 때문에 시간이 부족하고 여러 가지 일들이 겹쳐져서 정말 지친다 어쩌면 좋을까 휴식을 취하면서 스트레스를 풀 수 있는 방법을 찾아봐야겠다', '0'], ['혹시 좋은 조언이 있을까', '0']]
['슬픔', '슬픔', '슬픔', '슬픔', '화남']
[['우울', '자살', '우울', '피로', '분노'], 1.0]




['슬픔', '슬픔', '슬픔', '슬픔', '화남']


In [ ]:
## test2. 우울 문장 없을 경우
user_input = '오늘은 햇빛이 환하게 비치는 아름다운 날이었습니다 나는 일어나자마자 신나게 노래를 틀고 춤을 추면서 하루를 시작했습니다 아침 식사 후에는 친구들과 함께 공원에 나가서 피크닉을 즐겼습니다 우리는 함께 먹을 음식을 각자 준비해서 가져와서 다양한 요리를 공유했습니다 즐거운 이야기와 웃음 속에서 시간이 흘러갔고, 우리는 미니 게임을 열었습니다 경쟁하면서도 서로를 응원하며 즐거운 시간을 보냈습니다'

depress = Analyze_fn_depress(model_output(depress_model ,user_input, tokenizer_d))
emotion = Analyze_fn_emotion(model_output(emotion_model ,user_input, tokenizer_e))
print(depress)
print("\n\n\n")
print(emotion)

[['오늘은 햇빛이 환하게 비치는 아름다운 날이었습니다', '0'], ['나는 일어나자마자 신나게 노래를 틀고 춤을 추면서 하루를 시작했습니다', '0'], ['아침 식사 후에는 친구들과 함께 공원에 나가서 피크닉을 즐겼습니다', '0'], ['우리는 함께 먹을 음식을 각자 준비해서 가져와서 다양한 요리를 공유했습니다', '0'], ['즐거운 이야기와 웃음 속에서 시간이 흘러갔고, 우리는 미니 게임을 열었습니다', '0'], ['경쟁하면서도 서로를 응원하며 즐거운 시간을 보냈습니다', '0']]
emotion : 자존감저하 일 확률 41.48 %
emotion : 일상 일 확률 42.03 %
emotion : 일상 일 확률 84.56 %
emotion : 식욕저하 일 확률 82.25 %
emotion : 일상 일 확률 61.76 %
emotion : 일상 일 확률 66.52 %
[['오늘은 햇빛이 환하게 비치는 아름다운 날이었습니다', '0'], ['나는 일어나자마자 신나게 노래를 틀고 춤을 추면서 하루를 시작했습니다', '0'], ['아침 식사 후에는 친구들과 함께 공원에 나가서 피크닉을 즐겼습니다', '0'], ['우리는 함께 먹을 음식을 각자 준비해서 가져와서 다양한 요리를 공유했습니다', '0'], ['즐거운 이야기와 웃음 속에서 시간이 흘러갔고, 우리는 미니 게임을 열었습니다', '0'], ['경쟁하면서도 서로를 응원하며 즐거운 시간을 보냈습니다', '0']]
['중립', '슬픔', '슬픔', '행복함', '슬픔', '행복함']
[['일상', '일상', '일상', '식욕저하', '일상', '일상'], 0.17]




['중립', '슬픔', '슬픔', '행복함', '슬픔', '행복함']


In [ ]:
import random
import pandas as pd

def playlist(selected_value):

    df = pd.read_csv(f'/content/drive/MyDrive/KOBERT/self_care/{selected_value}.csv', encoding='utf-8')

    # 'href' 열에서 아무거나 하나 랜덤으로 선택
    random_value = random.choice(df['href'])

    # 선택된 값을 문자열로 반환
    selected_plyst = str(random_value)

    print("선택된 값:", selected_plyst)
    return selected_plyst

In [ ]:
lst = '우울'
playlist(lst)

선택된 값: https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=439184450


'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=439184450'

In [ ]:
import pandas as pd
def todolist(selected_value):

    df = pd.read_csv('/content/drive/MyDrive/KOBERT/self_care/todolist.csv', encoding='utf-8')

    # 특정 열의 값이 "selected_value"인 행 필터링
    selected_rows = df[df['depress'] == selected_value]

    # 필터링된 행이 있는 경우
    if not selected_rows.empty:
        # 필터링된 행 중에서 무작위로 하나 선택
        random_row = selected_rows.sample(n=1)
        selected_value = random_row['todo'].values[0]
        print("선택된 값:", selected_value)
    else:
        selected_value = "잘 하고 있어요! 즐거운 하루 보내세요~! "
        print("선택된 값이 없습니다.")

    return selected_value

In [ ]:
lst = '불'
todolist(lst)

선택된 값이 없습니다.


'잘 하고 있어요! 즐거운 하루 보내세요~! '

##Flask API 서버 실행

In [ ]:
!pip install flask
!pip install pyngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!pip install flask jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=640bd8e02565296ce1884bdf64c87698ac3d264c94b497b182aaa7e018fb787f
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
--2023-06-20 00:04:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-a

In [ ]:
# ngrok 개인 인증 키
!ngrok authtoken '2R5SgEswEFcWgKkawrdvZd3O7Yc_7QyjWgTEwHfUfGaVGnLNS'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, jsonify, request, render_template, make_response
from pyngrok import ngrok

app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
ngrok.kill() # 초기화

@app.route('/', methods=['GET','POST'])
def get_result():
    if request.method == 'POST':

        data = request.get_json()
        user_input = data['user_input']
        depress = Analyze_fn_depress(model_output(depress_model ,user_input, tokenizer_d))
        emotion = Analyze_fn_emotion(model_output(emotion_model ,user_input, tokenizer_e))

        return jsonify({'emotion_list' : emotion, 'depress_list' : depress[0], 'depress_value' : depress[1]})

    else:
        response = make_response('This is a GET request.')
        response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
        return response

@app.route('/playlist', methods=['GET','POST'])
def get_playlist():
    if request.method == 'POST':

        user_data = request.get_json()
        user_depress = user_data['depressName']
        user_playlist = playlist(user_depress)

        return jsonify({'user_playlist' : user_playlist})

    else:
        response = make_response('This is a GET request.')
        response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
        return response

@app.route('/todo', methods=['GET','POST'])
def todo():
    if request.method == 'POST':

        user_data = request.get_json()
        user_depress = user_data['depressName']
        todo = todolist(user_depress)

        return jsonify({'todo' : todo})

    else:
        response = make_response('This is a GET request.')
        response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
        return response

def start_ngrok():
    ngrok_tunnel_url = ngrok.connect(80)
    print('Public URL:', ngrok_tunnel_url)

if __name__ == '__main__':
    start_ngrok()
    app.run(port=80)

Public URL: NgrokTunnel: "https://e7ad-35-185-42-206.ngrok-free.app" -> "http://localhost:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit


[['오늘도 삶이 재미가 없고 우울하다 기분이 항상 다운 되어 있어서 강한 축 처지는 느낌이다', '0'], ['모든 것이 귀찮고 아무것도 하기 싫고 가만히 있고 싶다', '0'], ['내 일이 너무 피곤하다', '0'], ['대로 두눈을 감고 뜨지 않았으면 좋겠다', '0'], ['내가 쓸모없는 거 같고 죽고 싶다', '0']]
emotion : 우울 일 확률 96.91 %
emotion : 무기력 일 확률 77.43 %
emotion : 피로 일 확률 97.26 %
emotion : 집중력저하 일 확률 27.22 %
emotion : 자살 일 확률 94.08 %
[['오늘도 삶이 재미가 없고 우울하다 기분이 항상 다운 되어 있어서 강한 축 처지는 느낌이다', '0'], ['모든 것이 귀찮고 아무것도 하기 싫고 가만히 있고 싶다', '0'], ['내 일이 너무 피곤하다', '0'], ['대로 두눈을 감고 뜨지 않았으면 좋겠다', '0'], ['내가 쓸모없는 거 같고 죽고 싶다', '0']]


INFO:werkzeug:127.0.0.1 - - [20/Jun/2023 00:18:16] "POST / HTTP/1.1" 200 -


['슬픔', '슬픔', '슬픔', '슬픔', '슬픔']
[['오늘도 삶이 재미가 없고 우울하다 기분이 영상 다운 되어 있어서 다 안고 축 처지는 느낌이다', '0'], ['모든 것이 귀찮고 아무것도 하기 싫고 밤 보고 싶어 내일 너무 피곤하다', '0'], ['이대로 두눈을 감고 되지 않았으면 좋겠다', '0'], ['내가 쓸모없고 쓸모없는 거 같고 듣고 싶다', '0']]
emotion : 우울 일 확률 96.56 %
emotion : 피로 일 확률 81.15 %
emotion : 집중력저하 일 확률 25.67 %
emotion : 절망 일 확률 96.08 %
[['오늘도 삶이 재미가 없고 우울하다 기분이 영상 다운 되어 있어서 다 안고 축 처지는 느낌이다', '0'], ['모든 것이 귀찮고 아무것도 하기 싫고 밤 보고 싶어 내일 너무 피곤하다', '0'], ['이대로 두눈을 감고 되지 않았으면 좋겠다', '0'], ['내가 쓸모없고 쓸모없는 거 같고 듣고 싶다', '0']]


INFO:werkzeug:127.0.0.1 - - [20/Jun/2023 00:22:24] "POST / HTTP/1.1" 200 -


['슬픔', '슬픔', '슬픔', '슬픔']
[['한국 일상이 너무 재미가 없다 기분이 안 좋다', '0'], ['왜 여기 없어', '0']]
emotion : 무기력 일 확률 50.89 %
emotion : 일상 일 확률 52.22 %
[['한국 일상이 너무 재미가 없다 기분이 안 좋다', '0'], ['왜 여기 없어', '0']]


INFO:werkzeug:127.0.0.1 - - [20/Jun/2023 00:23:20] "POST / HTTP/1.1" 200 -


['중립', '화남']
